# Error analysis on consistency
- inspect cases where w/o an anchor model goes wrong, while with an anchor the model gave correct predictions

In [1]:
import json
import pandas as pd 
from collections import Counter, defaultdict 
from utils_path import dataset_to_respath
from inflection import singularize, pluralize 

pd.set_option('display.max.columns', 100)
pd.set_option('display.max.colwidth', 500)

def get_dataset_to_respath(dataset_to_respath, print_flag=False):
    # remote path 
    source_dir = 'spartan:~/cogsci/DAP/'
    target_dir = '../../'
    dataset_to_localpath = defaultdict()
    dataset_rename = {
        'hypernymsuite-BLESS': 'BLESS', 'lm_diagnostic_extended-singular': 'DIAG', 'clsb-singular':'CLSB', 'hypernymsuite-LEDS': 'LEDS', 'hypernymsuite-EVAL': 'EVAL', 'hypernymsuite-SHWARTZ': 
        "SHWARTZ"
    }
    dataset_name_to_relpath = defaultdict()
    for dataset, path in dataset_to_respath.items():
        path = path.replace(".tsv", ".csv")
        source_path = source_dir + path 
        dataset_l1 = dataset.split("-")[0]
        dataset_l2 = dataset.split("-")[1] 
        target_path = target_dir + path
        
        scp_string = f"!scp {source_path} {target_path}"
        if print_flag:
            print(scp_string)
            print()
#         print(target_path)
        dataset_to_localpath[dataset_rename[dataset]] = target_path 
        dataset_name_to_relpath[dataset_rename[dataset]] = "/".join(dataset.split('-'))
#     print(dataset_to_localpath)
    return dataset_to_localpath, dataset_name_to_relpath

dataset_to_localpath, dataset_name_to_relpath = get_dataset_to_respath(dataset_to_respath)
print(json.dumps(dataset_to_localpath, indent=4))
# print(dataset_name_to_relpath)

def read_anchors(path_pl, anchor_source, debug=False):
    '''
    read the anchor files mined from singualr and plural
    
    args: 
        anchor_soure: using the anchors mined from singular probe or plural probe
        
    return: 
        dic_sub_to_anchors_singular: both sub_label and subj_anchors are singular 
        dic_sub_to_anchors_plural: both sub_label and subj_anchors are plural 
    '''
#     dfsg = pd.read_csv(path_sg)
#     dfsg['subj_anchors'] = dfsg['subj_anchors'].apply(lambda x: eval(x))
    
#     dfpl = pd.read_csv(path_pl)
#     dfpl['subj_anchors'] = dfpl['subj_anchors'].apply(lambda x: eval(x))
#     df = pd.merge(dfsg, dfpl, on = 'uuid', suffixes=('_sg', '_pl'))
# if anchor_source == 'plural':
#         #convert the singular anchors into singular format
#         df['subj_anchors_sg'] = df['subj_anchors_pl'].progress_apply(lambda x: [singularize(word) for word in x])
#     elif anchor_source == 'singular':
#         #convert the plural anchors into singular format
#         df['subj_anchors_pl'] = df['subj_anchors_sg'].progress_apply(lambda x: [pluralize(word) for word in x])

    df = pd.read_csv(path_pl)
    
    if debug: df = df.head(5)
    df['subj_anchors_sg'] = df['subj_anchors_sg'].apply(lambda x: eval(x))
    df['subj_anchors_pl'] = df['subj_anchors_pl'].apply(lambda x: eval(x))
        
    dic_sub_to_anchors_singular = dict(zip(df['sub_label_sg'], df['subj_anchors_sg']))
    dic_sub_to_anchors_plural = dict(zip(df['sub_label_pl'], df['subj_anchors_pl']))
    
    return dic_sub_to_anchors_singular, dic_sub_to_anchors_plural



 

{
    "BLESS": "../../log/bert-large-uncased/hypernymsuite/BLESS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.HYPERNYMSUITE.csv",
    "DIAG": "../../log/bert-large-uncased/lm_diagnostic_extended/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.LM_DIAGNOSTIC_EXTENDED.csv",
    "CLSB": "../../log/bert-large-uncased/clsb/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False.CLSB.csv",
    "LEDS": "../../log/bert-large-uncased/hypernymsuite/LEDS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_fil

In [2]:
from utils_path import dataset_to_respath
dataset_to_localpath, dataset_name_to_relpath = get_dataset_to_respath(dataset_to_respath)

# for dataset, data_path in dataset_to_localpath.items():
#     dic_sub_to_anchors_singular,dic_sub_to_anchors_plural = read_anchors(data_path, data_path, anchor_source='plural', debug=False)


def concept_evaluation(label, pred):
    '''
    
    label: a list with the singualr and plural labels (e.g., ['tool', 'tools'])
    pred: the top K prediction list 

    return:
        1 if label share with pred else 0  
    '''
    if not isinstance(label, list):
        # label = eval(label)
        label = [label]
        
    if not isinstance(pred, list):
        print(type(pred), pred)
        pred = eval(pred)

    shared = set(label).intersection(set(pred))
    return 1 if len(shared)>0 else 0 
    

## Pair Consistency

In [3]:

from scipy import stats 
dataset_to_respath={
    "CLSB": "../../log/bert-large-uncased/clsb/singular/consistency/",
    "BLESS": "../../log/bert-large-uncased/hypernymsuite/BLESS/consistency/",
    "EVAL": "../../log/bert-large-uncased/hypernymsuite/EVAL/consistency/",
    "LEDS": "../../log/bert-large-uncased/hypernymsuite/LEDS/consistency/",
    "DIAG": "../../log/bert-large-uncased/lm_diagnostic_extended/singular/consistency/",
    "SHWARTZ": "../../log/bert-large-uncased/hypernymsuite/SHWARTZ/consistency/"
}


out_cols_pl = ['sub_label_singular', 'obj_label_singular',
               'obj_mask_sentence_pl_1', 'obj_mask_sentence_pl_2',
       'obj_mask_sentence_pl_3', 'obj_mask_sentence_pl_4',
       'obj_mask_sentence_pl_5', 'obj_mask_sentence_pl_6', 
       'p1_pl_1','p1_pl_2',  'p1_pl_3', 
       'p1_pl_4', 'p1_pl_5', 'p1_pl_6', 'p1_pl', 'subj_anchors_pl',
       'mask_sentences_plural_1', 'mask_sentences_plural_2',
       'mask_sentences_plural_3', 'mask_sentences_plural_4',
       'mask_sentences_plural_5', 'mask_sentences_plural_6', 
                'sub_label_plural', 'obj_label_plural', 'uuid', 'relation',
       ]
#  'obj_mask_sentence_pl_1', 'obj_mask_sentence_pl_2',
#        'obj_mask_sentence_pl_3', 'obj_mask_sentence_pl_4',
#        'obj_mask_sentence_pl_5', 'obj_mask_sentence_pl_6', 


out_cols_sg = ['sub_label_singular', 'obj_label_singular',
       'obj_mask_sentence_sg_1', 'obj_mask_sentence_sg_2', 'obj_mask_sentence_sg_3', 
       'p1_sg_1','p1_sg_2',  'p1_sg_3', 'p1_sg', 'subj_anchors_sg',
       'mask_sentences_singular_1', 'mask_sentences_singular_2', 'mask_sentences_singular_3',
       'sub_label_plural', 'obj_label_plural', 'uuid', 'relation',
              ]

min_pvalue = 0.05
df_res_all = []
col_gains = []
for mask_type in [['lsp_sap', 'lsp_dap'], ['def_sap', 'def_dap']]: 
# for mask_type in [['lsp_sap', 'lsp_dap']]: #, ]: #, 
    print(mask_type)
    pattern_num = 6 if 'lsp' in mask_type[0] else 3
    metric = 'p1_pl' if 'lsp' in mask_type[0] else 'p1_sg'
    out_cols = out_cols_pl if 'lsp' in mask_type[0] else out_cols_sg
    
    for dataset, path in dataset_to_respath.items():
        #if dataset!='CLSB': continue 
        dic_sub_to_anchors_singular, dic_sub_to_anchors_plural = read_anchors(dataset_to_localpath[dataset] , anchor_source='plural', debug=False)
        
        suffix1 = f'IsA.{mask_type[0]}.csv'
        suffix2 = f'IsA.{mask_type[1]}.csv'

        df1 = pd.read_csv(path+suffix1).sort_values(by='uuid')
        #display(df1)
        
#         df1['subj_anchors_sg'] = df1['sub_label_singular'].apply(lambda x: dic_sub_to_anchors_singular.get(x))
#         df1['subj_anchors_pl'] = df1['sub_label_plural'].apply(lambda x: dic_sub_to_anchors_plural.get(x))
#         display(df1)
        
#         df2 = pd.read_csv(path+suffix2).sort_values(by='uuid')
       
#         statistic, pvalue = stats.ttest_rel(df1[metric], df2[metric])
#         reject_np = True if pvalue < min_pvalue else False
#         print(dataset, reject_np) #, statistic, pvalue, len(df1.index), len(df2.index))
        
#         df2['subj_anchors_sg'] = df2['sub_label_singular'].apply(lambda x: dic_sub_to_anchors_singular.get(x))
#         df2['subj_anchors_pl'] = df2['sub_label_plural'].apply(lambda x: dic_sub_to_anchors_plural.get(x))
        
#         df1[metric+'_anchor'] = df2[metric]
#         df2[metric+'_no_anchor'] = df1[metric]
            
#         #display(df1.columns)
#         #display(df1[['sub_label_plural', 'obj_label_plural','p1_sgpl', 'uuid' ]])
#         #display(df2[['sub_label_plural', 'obj_label_plural','p1_sgpl', 'uuid' ]])
#         dfg=df1.query(f"{metric}==0")#[['sub_label_plural', 'obj_label_plural', 'subj_anchors_pl']] #, 'subj_anchors']]
       
#         group_cols = ['sub_label_plural', 'obj_label_plural'] if 'lsp' in mask_type[0] else ['sub_label_singular', 'obj_label_singular']
#         for name, group in dfg.groupby(group_cols):
#             #display(group.col)
#             df2g =df2.query(f"{group_cols[0]}=='{name[0]}' and {group_cols[1]}=='{name[1]}' and {metric}==1")
#             if len(df2g.index)>0:
#                 df1q = df1.query(f"{group_cols[0]}=='{name[0]}' and {group_cols[1]}=='{name[1]}'")[out_cols]
#                 df2q = df2g[out_cols]
#                 display(pd.concat([df1q, df2q]))
#         cols= ['p1_pl_1', 'p1_pl_2', 'p1_pl_3', 'p1_pl_4', 'p1_pl_5', 'p1_pl_6'] if 'lsp' in mask_type[0] else ['p1_sg_1', 'p1_sg_2', 'p1_sg_3']
        
#         for col in cols:
#             grain_ratio = 0
#             for y1, y2 in zip(df1[col], df2[col]):
#                 if y1==0 and y2==1: grain_ratio +=1
#             grain_ratio = grain_ratio/len(df1.query(f"{col}==0").index)
#             #print(col, grain_ratio)
#             col_gains.append({'dataset': dataset, 'mask_type': mask_type[0], 'col':col, 'gains_from_anchor': grain_ratio})
# #         display("-"*40)
# #     print("-"*80)
# col_gains = pd.DataFrame(col_gains)
# display(col_gains)

['lsp_sap', 'lsp_dap']
['def_sap', 'def_dap']


## Group Consistency

In [9]:

from scipy import stats 
dataset_to_respath={
    "CLSB": "../../log/bert-large-uncased/clsb/singular/consistency_group/",
    "BLESS": "../../log/bert-large-uncased/hypernymsuite/BLESS/consistency_group/",
    "EVAL": "../../log/bert-large-uncased/hypernymsuite/EVAL/consistency_group/",
    "LEDS": "../../log/bert-large-uncased/hypernymsuite/LEDS/consistency_group/",
    "DIAG": "../../log/bert-large-uncased/lm_diagnostic_extended/singular/consistency_group/",
    "SHWARTZ": "../../log/bert-large-uncased/hypernymsuite/SHWARTZ/consistency_group/"
}


out_cols_pl = ['sub_label_singular', 'obj_label_singular',
               'obj_mask_sentence_pl_1', 'obj_mask_sentence_pl_2',
       'obj_mask_sentence_pl_3', 'obj_mask_sentence_pl_4',
       'obj_mask_sentence_pl_5', 'obj_mask_sentence_pl_6', 
       'p1_pl_1','p1_pl_2',  'p1_pl_3', 
       'p1_pl_4', 'p1_pl_5', 'p1_pl_6', 'p1_pl', 'subj_anchors_pl',
       'mask_sentences_plural_1', 'mask_sentences_plural_2',
       'mask_sentences_plural_3', 'mask_sentences_plural_4',
       'mask_sentences_plural_5', 'mask_sentences_plural_6', 
                'sub_label_plural', 'obj_label_plural', 'uuid', 'relation',
       ]
#  'obj_mask_sentence_pl_1', 'obj_mask_sentence_pl_2',
#        'obj_mask_sentence_pl_3', 'obj_mask_sentence_pl_4',
#        'obj_mask_sentence_pl_5', 'obj_mask_sentence_pl_6', 


out_cols_sg = ['sub_label_singular', 'obj_label_singular',
       'obj_mask_sentence_sg_1', 'obj_mask_sentence_sg_2', 'obj_mask_sentence_sg_3', 
       'p1_sg_1','p1_sg_2',  'p1_sg_3', 'p1_sg', 'subj_anchors_sg',
       'mask_sentences_singular_1', 'mask_sentences_singular_2', 'mask_sentences_singular_3',
       'sub_label_plural', 'obj_label_plural', 'uuid', 'relation',
              ]


def evaluate(df, pattern_num, mask_type, dataset):
    df_res=[]
    for k in [10]:
        # for i, (pred_col_sg, pred_col_pl) in enumerate(zip(pred_col_sg, pred_col_pl), start=1):
        for i in range(1, pattern_num+1):
            df[f'p1_sg_{i}'] = df[['obj_label_singular', f'obj_mask_sentence_sg_{i}']].apply(lambda x: concept_evaluation(x[0], eval(x[1])[:k]), axis=1 )
            df[f'p1_pl_{i}'] = df[['obj_label_plural', f'obj_mask_sentence_pl_{i}']].apply(lambda x: concept_evaluation(x[0], eval(x[1])[:k]), axis=1 )

        pred_col_sg_p1 = [x for x in df.columns if 'p1_sg_' in x ]
        pred_col_pl_p1 = [x for x in df.columns if 'p1_pl_' in x ]


        df['p1_sg'] = df[pred_col_sg_p1].apply(lambda x: int(all(ele == 1 for ele in x)), axis=1)
        df['p1_pl'] = df[pred_col_pl_p1].apply(lambda x: int(all(ele == 1 for ele in x)), axis=1)

        df['p1_sgpl'] = df[['p1_sg', 'p1_pl']].apply(lambda x: 1 if x[0]==1 and x[1]==1 else 0, axis=1)

        acc_sg = round(df['p1_sg'].sum()/len(df.index) * 100, 1)
        acc_pl = round(df['p1_pl'].sum()/len(df.index)* 100, 1) 
        acc_sgpl = round(df['p1_sgpl'].sum()/len(df.index)* 100, 1) 
        
        acc_consistency = acc_pl if 'lsp' in mask_type else acc_sg 
        df_res.append({"dataset":dataset, "mask_type":mask_type, "K":k, 'Consistency': acc_consistency})
        #df_res.append({"K":k, 'Singular': acc_sg, 'Plural': acc_pl, 'Paired Singular-Plural': acc_sgpl})
    print(df_res)
    return  df, df_res 
    

min_pvalue = 0.05
df_res_all = []
col_gains = []
vocab_hyper = set()
vocab_hyper_wrong_all = set()
vocab_hyper_all = set()
dfs_no_anchor = []
dfs_anchor = []
# for mask_type in [['lsp_sap', 'lsp_dap'], ['def_sap', 'def_dap']]: 
for mask_type in [['lsp_sap', 'lsp_dap']]: #, ]: #, 
# for mask_type in [['def_sap', 'def_dap']]: #, ]: #, 
    print(mask_type)
    pattern_num = 6 if 'lsp' in mask_type[0] else 3
    metric = 'p1_pl' if 'lsp' in mask_type[0] else 'p1_sg'
    out_cols = out_cols_pl if 'lsp' in mask_type[0] else out_cols_sg
    
    for dataset, path in dataset_to_respath.items():
        #if dataset!='CLSB': continue 
        dic_sub_to_anchors_singular, dic_sub_to_anchors_plural = read_anchors(dataset_to_localpath[dataset] , anchor_source='plural', debug=False)
        
        suffix1 = f'IsA.{mask_type[0]}.csv'
        suffix2 = f'IsA.{mask_type[1]}.csv'

        df1 = pd.read_csv(path+suffix1).sort_values(by='uuid')
        df1['subj_anchors_sg'] = df1['sub_label_singular'].apply(lambda x: dic_sub_to_anchors_singular.get(x))
        df1['subj_anchors_pl'] = df1['sub_label_plural'].apply(lambda x: dic_sub_to_anchors_plural.get(x))
        df1['obj_label_singular'] = df1['obj_label_singular'].apply(lambda x: eval(x)[0])
        vocab_hyper_all.update(set(df1['obj_label_singular']))
        
        df2 = pd.read_csv(path+suffix2).sort_values(by='uuid')
        
        
        df2['subj_anchors_sg'] = df2['sub_label_singular'].apply(lambda x: dic_sub_to_anchors_singular.get(x))
        df2['subj_anchors_pl'] = df2['sub_label_plural'].apply(lambda x: dic_sub_to_anchors_plural.get(x))
        df2['obj_label_singular'] = df2['obj_label_singular'].apply(lambda x: eval(x)[0])
       
        df1[metric+'_anchor'] = df2[metric]
        df2[metric+'_no_anchor'] = df1[metric]
            
        dfs_no_anchor.append(df1)
        dfs_anchor.append(df2)
        
dfs_no_anchor = pd.concat(dfs_no_anchor)
dfs_anchor = pd.concat(dfs_anchor)


['lsp_sap', 'lsp_dap']


## Depth in WordNet and their consistency

In [23]:
# query = ['sport', 'beverage', 'sweet', 'garment']

df_word_depth = pd.read_csv('../log/word_depth_wn.csv')
df_results =[]
for name, group in df_word_depth.groupby('depth'):
    query = list(set(group['word']))
    dfq1 = dfs_no_anchor.query(f'obj_label_singular in {query}')
    dfq2 = dfs_anchor.query(f'obj_label_singular in {query}')
    consistency1 = round(dfq1['p1_pl'].mean()*100, 1)
    consistency2 = round(dfq2['p1_pl'].mean()*100, 1)
    gains = round(consistency2-consistency1, 1)
    examples = dict(Counter(dfq1['obj_label_singular'].values).most_common(5)).keys()
    #print(name, len(dfq1), len(dfq1.query('p1_pl==0').index)/len(dfq1.index), len(dfq2.query('p1_pl==0').index)/len(dfq2.index))
    print(name, len(dfq1), consistency1,consistency2,  )
#     print(name, len(dfq2), )
    df_results.append({'depth': name, '#instances': len(dfq1.index), r'\lsp': consistency1, r'lspa': consistency2, "gains": gains, 'examples':examples })
df_word_depth = pd.read_csv('../log/word_depth_wn.csv')
query_depth = [7] #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17]
print( set(df_word_depth.query(f'depth in {query_depth}')['word']))
print(pd.DataFrame(df_results).to_latex(index=False, escape=False))

1 5 20.0 20.0
2 104 1.0 7.7
3 352 2.8 3.7
4 1335 8.5 12.2
5 1572 6.7 11.1
6 4829 8.4 11.5
7 1017 29.7 41.5
8 1773 9.4 14.8
9 1110 32.0 36.2
10 348 28.2 32.5
11 15 6.7 6.7
12 12 16.7 25.0
13 55 5.5 9.1
14 54 7.4 7.4
16 2 50.0 100.0
17 2 0.0 0.0
{'jewelry', 'constellation', 'payment', 'soil', 'crop', 'movement', 'service', 'stick', 'software', 'accessory', 'prejudice', 'wall', 'grape', 'science', 'gene', 'country', 'sensation', 'state', 'school', 'article', 'instrument', 'house', 'berry', 'satellite', 'pet', 'deity', 'rule', 'vessel', 'suburb', 'box', 'letter', 'clothes', 'vehicle', 'criminal', 'cosmetic', 'dish', 'tool', 'church', 'plant', 'enclosure', 'garment', 'finger', 'predator', 'sweet', 'snack', 'skirt', 'passageway', 'bishop', 'restaurant', 'hope', 'site', 'hormone', 'furniture', 'hair', 'highway', 'craft', 'prophet', 'protest', 'limb', 'female', 'sport', 'machine'}
\begin{tabular}{rrrrrl}
\toprule
 depth &  #instances &  \lsp &  lspa &  gains &                                  

<ipython-input-23-440279232b8f>:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(df_results).to_latex(index=False, escape=False))


In [12]:
set1 = set (dfs_no_anchor.query("p1_pl == 1")['obj_label_singular'] ) 
set2 = set (dfs_anchor.query("p1_pl == 1")['obj_label_singular'] ) 
set2 - set1 

{'agreement',
 'airplane',
 'alcohol',
 'algae',
 'art',
 'artist',
 'battle',
 'carriage',
 'change',
 'citizen',
 'competition',
 'continent',
 'count',
 'county',
 'court',
 'digit',
 'facility',
 'finger',
 'genre',
 'gift',
 'group',
 'hair',
 'hormone',
 'implement',
 'jewelry',
 'letter',
 'literature',
 'meat',
 'memorial',
 'month',
 'monument',
 'noise',
 'pattern',
 'pest',
 'politician',
 'proceeding',
 'prophet',
 'publisher',
 'question',
 'rapid',
 'relative',
 'saint',
 'seed',
 'sibling',
 'snack',
 'soil',
 'spice',
 'suburb',
 'sweet',
 'symbol',
 'vampire',
 'variety',
 'waste',
 'wetland'}

In [33]:
# dfs_no_anchor.head()
# dfs_no_anchor.query()
for col in ['p1_pl_1', 'p1_pl_2', 'p1_pl_3', 'p1_pl_4', 'p1_pl_5', 'p1_pl_6']:
    numerator = dfs_no_anchor.query(f"{col}==0") #, dfs[dfs_anchor]
    denominator = dfs_no_anchor.query(f"p1_pl==0")
    ratio = len(numerator.index)/ len(denominator.index)
    print(col, ratio)

p1_pl_1 0.8057103064066853
p1_pl_2 0.897125854646746
p1_pl_3 0.9213724993669283
p1_pl_4 0.8503418586984046
p1_pl_5 0.8502152443656622
p1_pl_6 0.865662192960243


In [53]:
count = 0 
out = dfs_no_anchor[['p1_pl_1', 'p1_pl_2', 'p1_pl_4', 'p1_pl_5', 'p1_pl_6',  'p1_pl_3']].apply(lambda x: 1 if int(all(ele == 1 for ele in x[:-1]))==1 and x[-1]==0 else 0, axis=1)
# for x in dfs_no_anchor.query("p1_pl==0")[['p1_pl_1', 'p1_pl_2', 'p1_pl_3', 'p1_pl_4', 'p1_pl_5', 'p1_pl_6']].iterrows():
#     break
wrong= sum(out)/len(dfs_no_anchor.query('p1_pl==0').index)
print(wrong)

0.010572296783995948


## Predicted hypernyms is Co-hyponyms

In [80]:
dfs_no_anchor.head()
df_cohypo = pd.read_csv('../log/word_to_cohyponyms.txt')
word_to_cohyponyms = dict(zip(df_cohypo['word'].to_list(), df_cohypo['cohyponyms'].to_list()))

dfs_no_anchor['sub_sister'] = dfs_no_anchor['sub_label_singular'].apply(lambda x: eval(word_to_cohyponyms.get(x)))

dfs_no_anchor['obj_mask_sentence_sg_1'] = dfs_no_anchor['obj_mask_sentence_sg_1'].apply(lambda x: eval(x))
def get_shared(x, y):
    shared = set(x).intersection(set(y))
    return 1 if len(shared)>0 else 0
dfs_no_anchor['obj_in_sub_sister'] = dfs_no_anchor[['obj_mask_sentence_sg_1', 'sub_sister']].apply(lambda x: get_shared(x[0], x[1]), axis=1)
# dfs_no_anchor[['sub_label_singular','obj_mask_sentence_sg_1']]

In [81]:
dfs_no_anchor['obj_in_sub_sister'].mean()

0.33977979348508186

{'predator', 'protest', 'service', 'sweet', 'constellation', 'school', 'house', 'movement', 'stick', 'restaurant', 'vehicle', 'skirt', 'berry', 'site', 'sensation', 'limb', 'clothes', 'hope', 'vessel', 'enclosure', 'article', 'payment', 'gene', 'highway', 'sport', 'bishop', 'wall', 'jewelry', 'software', 'snack', 'cosmetic', 'grape', 'box', 'state', 'plant', 'suburb', 'female', 'prophet', 'criminal', 'craft', 'country', 'rule', 'church', 'science', 'pet', 'hair', 'instrument', 'satellite', 'tool', 'garment', 'finger', 'passageway', 'machine', 'soil', 'furniture', 'crop', 'prejudice', 'dish', 'accessory', 'hormone', 'deity', 'letter'}


In [16]:
word_to_depth

,Unnamed: 0,word,depth
0,267,conflict,1
1,249,transaction,1
2,329,proceeding,2
3,205,relation,2
4,239,group,2
...,...,...,...
327,139,pest,13
328,188,human,13
329,324,horse,14
330,176,cattle,16


In [ ]:
import re 
import wn, wn.taxonomy
ewn = wn.Wordnet('ewn:2020')


results = defaultdict()
vocab_depths = defaultdict()
names = [ 'wrong_hyper_no_anchor', 'corrcted_by_anchor', 'all_hyper',]
vocabs = [vocab_hyper_wrong_all, vocab_hyper, vocab_hyper_all] 
res_oov = defaultdict()
for name, vocab in zip(names, vocabs):
    vocab_depth = defaultdict()
    vocab_oov  =  set()
    print(len(vocab))
    for word in vocab: #['concept', 'thought', 'living thing', 'whole', 'psychological feature', 'unit', 'artifact', 'abstraction', 'object','physical entity', 'entity']:
        #word = eval(word)[0] if 
        synset = wn.synsets(word, pos='n')
        if len(synset)==0:
            vocab_oov.add(word)
            continue 
            
        synset = synset[0]
        min_depth = wn.taxonomy.min_depth(synset, simulate_root=False)
        #print(word, min_depth)
        vocab_depth[word] = min_depth

    print(Counter(vocab_depth.values()).most_common())
    df_depth = pd.DataFrame(vocab_depth.items(), columns=['word', 'depth']).sort_values('depth')
    query_depth = [5, 6, 7, 8, 9]
    display(df_depth.query(f"depth in {query_depth}").sample(5))
    results[name] = df_depth
    res_oov[name] = vocab_oov
    vocab_depths[name] = vocab_depth

In [ ]:
from matplotlib import pyplot as plt
dict1 = dict(results['corrcted_by_anchor']['depth'].value_counts())
dict2 = dict(results['wrong_hyper_no_anchor']['depth'].value_counts())
dict3 = dict(results['all_hyper']['depth'].value_counts())

df_correct = []
for k,v in dict1.items():
    df_correct.append({'depth':k, 'corrected_ratio': round(v/dict2[k], 3) ,'wrong': dict2[k], 'correct': v, 'all': dict3[k]})
print("-"*80)
df_correct = pd.DataFrame(df_correct)
df_correct.sort_values(['wrong', 'corrected_ratio'], ascending=False)
# display(results['all_hyper']['depth'].value_counts(normalize=True).plot(kind='bar', title='distribution overall'))
# plt.show()
# display(results['wrong_hyper_no_anchor']['depth'].value_counts(normalize=True).plot(kind='bar', title='distribution of wrong w/o anchors'))
# plt.show()

In [ ]:
# dict1 = dict(results['corrcted_by_anchor']['depth'].value_counts())
# dict2 = dict(results['wrong_hyper_no_anchor']['depth'].value_counts())
# dict3 = dict(results['all_hyper']['depth'].value_counts())
query_depth = [5, 6, 7, 8]
dfq =results['corrcted_by_anchor'].query(f'depth in {query_depth}')
for name, group in dfq.groupby('depth'):
    display(name, group)

In [ ]:
results['corrcted_by_anchor']